In [1]:
import glob
import re
import pandas as pd

In [2]:
data_dir = '../data/'

In [3]:
files = glob.glob(data_dir + 'etl_s2_v2*.parquet')
files

['../data/etl_s2_v2_1680419520_1680590400.parquet',
 '../data/etl_s2_v2_1680112320_1680265920.parquet',
 '../data/etl_s2_v2_1679805120_1679958720.parquet',
 '../data/etl_s2_v2_1679958720_1680112320.parquet',
 '../data/etl_s2_v2_1680590400_1680744000.parquet',
 '../data/etl_s2_v2_1679178240_1679331840.parquet',
 '../data/etl_s2_v2_1680265920_1680419520.parquet',
 '../data/etl_s2_v2_1679485440_1679651520.parquet',
 '../data/etl_s2_v2_1679651520_1679805120.parquet',
 '../data/etl_s2_v2_1679331840_1679485440.parquet']

In [4]:
tickers = ['BTC-USD','ETH-USD','MKR-USD','BCH-USD','COMP-USD','AAVE-USD','UNI-USD','CRV-USD','BAL-USD','LTC-USD','ADA-USD','XLM-USD','ALGO-USD','ZEC-USD','DOGE-USD','SHIB-USD','SUSHI-USD','EOS-USD','ETC-USD','WBTC-USD']

In [221]:
ticker = 'ETH-USD'

In [222]:
data = pd.read_parquet(files[0])
columns = data.columns

In [223]:
def get_ticker_columns(ticker, columns, const_columns):
    cols = [col for col in columns if col.startswith(ticker) or col in const_columns]
    return cols

In [224]:
def get_column_field(column, const_columns):
    if column in const_columns:
        return column
    m = re.match(r'[a-zA-Z-]+:(.*)$', column)
    return m.group(1)

In [225]:
def get_all_data(files, columns, ticker, const_columns):
    tcolumns = get_ticker_columns(ticker, columns, const_columns)
    colmap = {col : get_column_field(col, const_columns) for col in tcolumns}
    dfs = []
    for file in files:
        df = pd.read_parquet(file, columns=tcolumns)
        df = df.rename(columns=colmap)
        dfs.append(df)
    df = pd.concat(dfs)
    df.sort_index(inplace=True)
    return df

In [226]:
data = get_all_data(files, columns, ticker, ['book_mean_return_27', 'sequence_interval_s'])

In [227]:
data.shape

(1530331, 710)

In [228]:
row_count = data.shape[0]

In [229]:
nas = data.isna().sum()

In [230]:
nas.sum()

7086553

In [231]:
threshold = 0.1

In [232]:
book_col_count = 0
trade_col_count = 0
for (count, name) in zip(nas, nas.index):
    nan_percent = float(count) / row_count
    if nan_percent > threshold:
        print(f"{name}: {count}")
    if nan_percent > threshold and 'trade' not in name:
        book_col_count += 1
    if nan_percent > threshold and 'trade' in name:
        trade_col_count += 1
print(f"total book column count: {book_col_count}")
print(f"total trade column count: {trade_col_count}")

ba_spread_960kurt: 163614
ba_spread_648kurt: 163614
ba_spread_648skew: 163614
ba_spread_162kurt: 163614
ba_spread_162skew: 163614
ba_spread_81kurt: 163614
ba_spread_9kurt: 163614
ba_spread_9skew: 163614
ba_spread_27kurt: 163614
trade_avg_price: 571639
ba_spread_27skew: 163614
ba_spread_81skew: 163614
ba_spread_3skew: 163614
ba_spread_324skew: 163614
ba_spread_960skew: 163614
ba_spread_3kurt: 163614
ba_spread_324kurt: 163614
trade_return: 571639
total book column count: 16
total trade column count: 2
